In [1]:
import tqdm
import torch
import torchvision

print(tqdm.__version__)
print(torch.__version__)
print(torchvision.__version__)

4.65.0
2.1.0
0.16.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

In [3]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=128
num_classes=10
num_epochs=20
lr=0.1

In [4]:
transform={
    "train":transforms.Compose([
        transforms.ToTensor(),
    ]),
    "test":transforms.Compose([
        transforms.ToTensor(),
    ])
}

dataset={
    "train":torchvision.datasets.CIFAR10(root="CIFAR10",train=True,transform=transform["train"],download=True),
    "test":torchvision.datasets.CIFAR10(root="CIFAR10",train=False,transform=transform["test"],download=True)
}

dataloader={
    "train":torch.utils.data.DataLoader(dataset["train"],batch_size=batch_size,shuffle=True,num_workers=2),
    "test":torch.utils.data.DataLoader(dataset["test"],batch_size=batch_size,shuffle=False,num_workers=2)
}


100%|██████████| 170498071/170498071 [00:12<00:00, 13654647.63it/s]


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10
Files already downloaded and verified


In [5]:
model=torchvision.models.resnet18(pretrained=False)
model.fc=nn.Linear(model.fc.in_features,num_classes)
model=model.to(device)

optimizer=optim.Adam(model.parameters(),lr=lr)
criterion=nn.CrossEntropyLoss()

/home/dnn2/anaconda3/envs/deep/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dnn2/anaconda3/envs/deep/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/dnn2/anaconda3/envs/deep/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
def train(model,dataloader,otpimizer,criterion,num_epochs,device):

    for epoch in range(1,num_epochs+1):

        for phase in ["train","test"]:

            if phase=="train":
                model.train()
            elif phase=="test":
                model.eval()

            with torch.set_grad_enabled(phase=="train"):
                loss_sum=0
                corrects=0
                total=0

                with tqdm(total=len(dataloader[phase]),unit="batch") as pbar:
                    pbar.set_description(f"Epoch[{epoch}/{num_epochs}]({phase})")
                    for imgs,labels in dataloader[phase]:   
                        imgs,labels=imgs.to(device),labels.to(device)
                        output=model(imgs)
                        loss=criterion(output,labels)

                        if phase=="train":
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()

                        predicted=torch.argmax(output,dim=1) ## dimあってる？
                        corrects+=(predicted==labels).sum()
                        total+=imgs.size(0)
                            
                        #loss関数で通してでてきたlossはCrossEntropyLossのreduction="mean"なので平均
                        #batch sizeをかけることで、batch全体での合計を今までのloss_sumに足し合わせる
                        loss_sum+=loss*imgs.size(0) 

                        accuracy=corrects.item()/total
                        running_loss=loss_sum/total
                        pbar.set_postfix({"loss":running_loss.item(),"accuracy":accuracy })
                        pbar.update(1)

In [7]:

train(model,dataloader,optimizer,criterion,num_epochs,device)

Epoch[20/20](test): 100%|██████████| 79/79 [00:00<00:00, 98.90batch/s, loss=1.19, accuracy=0.67]  
